In [ ]:
'''
Chunk parsing, also known as partial parsing, light parsing, or just chunking,
is an approach in which the parser assigns incomplete syntactic structure to the phrase.
The most common use of chunking is in information extraction and message understanding,
where the content of a text is mined for information that will be used to fill out a template. 
Chunk parsing is actually more like tagging than conventional parsing, in the sense that it typically 
uses finite-state methods and employs rules that work directly off the (tagged) surface string. 
This contrasts with conventional parsing, which normally uses context-free rules expressed over 
abstract categories (e.g. phrasal categories).

In terms of the other NLP tasks, chunking usually takes place after "tokenization and tagging."
'''

In [1]:
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

In [2]:
train_text = state_union.raw("2005-GWBush.txt")
sample_text = state_union.raw("2006-GWBush.txt")

In [3]:
custom_sent_tokenizer = PunktSentenceTokenizer(train_text)

In [4]:
tokenized = custom_sent_tokenizer.tokenize(sample_text)

In [5]:
def process_content():
    try:
        for i in tokenized:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            
            chunk_gram = r"""Chunk: {<RB.?>*<VB.?>*<NNP>+<NN>?}"""
            
            chunk_parser = nltk.RegexpParser(chunk_gram)
            chunked = chunk_parser.parse(tagged)
            
            print(chunked)
            
            ## UNCOMMENT AT YOUR OWN RISK :(
            # chunked.draw()
    except Exception as e:
        print(str(e))

In [6]:
process_content()

(S
  (Chunk PRESIDENT/NNP GEORGE/NNP W./NNP BUSH/NNP)
  'S/POS
  (Chunk ADDRESS/NNP)
  BEFORE/IN
  (Chunk A/NNP JOINT/NNP SESSION/NNP)
  OF/IN
  (Chunk THE/NNP CONGRESS/NNP ON/NNP THE/NNP STATE/NNP)
  OF/IN
  (Chunk THE/NNP UNION/NNP January/NNP)
  31/CD
  ,/,
  2006/CD
  (Chunk THE/NNP PRESIDENT/NNP)
  :/:
  (Chunk Thank/NNP)
  you/PRP
  all/DT
  ./.)
(S
  (Chunk Mr./NNP Speaker/NNP)
  ,/,
  (Chunk Vice/NNP President/NNP Cheney/NNP)
  ,/,
  members/NNS
  of/IN
  (Chunk Congress/NNP)
  ,/,
  members/NNS
  of/IN
  the/DT
  (Chunk Supreme/NNP Court/NNP)
  and/CC
  diplomatic/JJ
  corps/NN
  ,/,
  distinguished/JJ
  guests/NNS
  ,/,
  and/CC
  fellow/JJ
  citizens/NNS
  :/:
  Today/VB
  our/PRP$
  nation/NN
  lost/VBD
  a/DT
  beloved/VBN
  ,/,
  graceful/JJ
  ,/,
  courageous/JJ
  woman/NN
  who/WP
  (Chunk called/VBD America/NNP)
  to/TO
  its/PRP$
  founding/NN
  ideals/NNS
  and/CC
  carried/VBD
  on/IN
  a/DT
  noble/JJ
  dream/NN
  ./.)
(S
  Tonight/NN
  we/PRP
  are/VBP
  comforted

In [ ]:
#Additional Stuff
'''
Training tagger based chunker
I’ve used the `conll2000` corpus for training chunker. conll2000 corpus defines the chunks using IOB tags.
It specifies where the chunk begins and ends, along with its types.
A POS tagger can be trained on these IOB tags
The chunk tags use the IOB format.
IOB : Inside,Outside,Beginning
B- prefix before a tag indicates, it’s the beginning of a chunk
I- prefix indicates that it’s inside a chunk
O- tag indicates the token doesn’t belong to any chunk
'''

In [8]:
#Here conll2000 corpus for training shallow parser model
nltk.download('conll2000')
from nltk.corpus import conll2000
data= conll2000.chunked_sents()
train_data=data[:10900]
test_data=data[10900:]
print(len(train_data),len(test_data))
print(train_data[1])

[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\susan\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


10900 48
(S
  Chancellor/NNP
  (PP of/IN)
  (NP the/DT Exchequer/NNP)
  (NP Nigel/NNP Lawson/NNP)
  (NP 's/POS restated/VBN commitment/NN)
  (PP to/TO)
  (NP a/DT firm/NN monetary/JJ policy/NN)
  (VP has/VBZ helped/VBN to/TO prevent/VB)
  (NP a/DT freefall/NN)
  (PP in/IN)
  (NP sterling/NN)
  (PP over/IN)
  (NP the/DT past/JJ week/NN)
  ./.)


In [ ]:
'''
tree2conlltags,conlltags2tree are chunking utility functions.
→ `tree2conlltags`, to get triples of (word, tag, chunk tags for each token). These tuples are then finally used to train a 
    tagger and it learns IOB tags for POS tags.
→ `conlltags2tree` to generate a parse tree from these token triples
Conlltags2tree() is reversal of tree2conlltags().We’ll be using these functions to train our parser
'''

In [9]:
from nltk.chunk.util import tree2conlltags,conlltags2tree
wtc=tree2conlltags(train_data[1])
wtc

[('Chancellor', 'NNP', 'O'),
 ('of', 'IN', 'B-PP'),
 ('the', 'DT', 'B-NP'),
 ('Exchequer', 'NNP', 'I-NP'),
 ('Nigel', 'NNP', 'B-NP'),
 ('Lawson', 'NNP', 'I-NP'),
 ("'s", 'POS', 'B-NP'),
 ('restated', 'VBN', 'I-NP'),
 ('commitment', 'NN', 'I-NP'),
 ('to', 'TO', 'B-PP'),
 ('a', 'DT', 'B-NP'),
 ('firm', 'NN', 'I-NP'),
 ('monetary', 'JJ', 'I-NP'),
 ('policy', 'NN', 'I-NP'),
 ('has', 'VBZ', 'B-VP'),
 ('helped', 'VBN', 'I-VP'),
 ('to', 'TO', 'I-VP'),
 ('prevent', 'VB', 'I-VP'),
 ('a', 'DT', 'B-NP'),
 ('freefall', 'NN', 'I-NP'),
 ('in', 'IN', 'B-PP'),
 ('sterling', 'NN', 'B-NP'),
 ('over', 'IN', 'B-PP'),
 ('the', 'DT', 'B-NP'),
 ('past', 'JJ', 'I-NP'),
 ('week', 'NN', 'I-NP'),
 ('.', '.', 'O')]

In [10]:
tree=conlltags2tree(wtc)
print(tree)

(S
  Chancellor/NNP
  (PP of/IN)
  (NP the/DT Exchequer/NNP)
  (NP Nigel/NNP Lawson/NNP)
  (NP 's/POS restated/VBN commitment/NN)
  (PP to/TO)
  (NP a/DT firm/NN monetary/JJ policy/NN)
  (VP has/VBZ helped/VBN to/TO prevent/VB)
  (NP a/DT freefall/NN)
  (PP in/IN)
  (NP sterling/NN)
  (PP over/IN)
  (NP the/DT past/JJ week/NN)
  ./.)


In [ ]:
'''
Why is Chunking Important
Chunking can be used for entity detection. It can be used to extract word patterns from a large text without necessarily
analyzing the entire text. Say you have a very large text and you only need to extract information such as name, item bought,
and price of the item and the date. You can define a chunk grammar and use chunking to detect this pattern and extract the 
information quickly. 

In summary, Chunking helps to identify and understand phrases in a sentence and can perform rapid word filtering based on some defined grammar.
'''